# Plot result maps
Plot the resuls (interpolated and error fields) from the netCDF created in notebook [interp_Calanus](interp_Calanus.ipynb).

In [ ]:
import os
import calendar
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature 
import cartopy.mpl.ticker as cartopyticker
import toml
coast = cfeature.GSHHSFeature(scale="high")
lon_formatter = cartopyticker.LongitudeFormatter()
lat_formatter = cartopyticker.LatitudeFormatter()
plt.style.use("calanus.mplstyle")

## Read configuration from config.ini

In [ ]:
config = toml.load("config.ini")
config

## Set Cartopy projection

In [ ]:
dom = config["domain"]
mainproj = ccrs.Mercator(central_longitude=0.5*(dom[0] + dom[1]),
    min_latitude=dom[2], max_latitude=dom[3])
datacrs = ccrs.PlateCarree();
mainproj

### Directories and files

In [ ]:
datadir = config["directories"]["datadir"]
figdir = config["directories"]["figdir"]
resdir = config["directories"]["resdir"]
resfigdir = config["directories"]["resfigdir"]
datafile = os.path.join(datadir, "MBA_CPRdata_Emodnet_21Jan22.csv")
woafile = os.path.join(datadir, "WOD/temperature_surface_WOD2.nc")
bathfile = os.path.join(datadir, "gebco_30sec_16.nc")
outputfile1 = os.path.join(resdir, "calanus_finmarchicus_year_month.nc")
outputfile2 = os.path.join(resdir, "calanus_helgolandicus_year_month.nc")
os.path.isfile(woafile)

## Read the fields and observations

In [ ]:
def read_results_netcdf(outputfile):
    with netCDF4.Dataset(outputfile) as nc:
        lon = nc.get_variables_by_attributes(standard_name="longitude")[0][:]
        lat = nc.get_variables_by_attributes(standard_name="latitude")[0][:]
        time = nc.get_variables_by_attributes(standard_name="time")[0]
        dates = netCDF4.num2date(time[:], time.units, only_use_cftime_datetimes=False)
        field = nc.variables["abundance"][:]
        error = nc.variables["error"][:]
        
    return lon, lat, dates, field, error

In [ ]:
lon1, lat1, dates1, field1, error1 = read_results_netcdf(outputfile1)
lon2, lat2, dates2, field2, error2 = read_results_netcdf(outputfile2)

## Plots

In [ ]:
resolution = str(config['dlon']).replace(".", "")
resolution

### Analysis
#### Canalus finmarchicus

In [ ]:
ntimes = len(dates1)
for ii in range(490, 510):
    
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111, projection=mainproj)
    pcm = ax.pcolormesh(lon1, lat1, field1[ii,:,:], shading='auto',
                       cmap=plt.cm.hot_r, vmin=0., vmax=25., transform=datacrs)

    cb = plt.colorbar(pcm, extend="max")

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
    ax.set_extent(dom)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title(f"Canalus finmarchicus\n{calendar.month_name[dates1[ii].month]} {dates1[ii].year}")
    
    figname = f"Canalus_finmarchicus_L{resolution}_{dates1[ii].year}{str(dates1[ii].month).zfill(2)}"
    plt.savefig(os.path.join(resfigdir, figname))
    plt.close()

#### Canalus helgolandicus

In [ ]:
for ii in range(490, 510):
    
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111, projection=mainproj)
    pcm = ax.pcolormesh(lon2, lat2, field2[ii,:,:], shading='auto',
                       cmap=plt.cm.hot_r, vmin=0., vmax=25., transform=datacrs)

    cb = plt.colorbar(pcm, extend="max")

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
    ax.set_extent(dom)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title(f"Canalus helgolandicus\n{calendar.month_name[dates1[ii].month]} {dates1[ii].year}")
    
    figname = f"Canalus_helgolandicus_L{resolution}_{dates1[ii].year}{str(dates1[ii].month).zfill(2)}"
    plt.savefig(os.path.join(resfigdir, figname))
    plt.close()

#### Error field
The field is the same for both species as the data distribution is the same.

In [ ]:
for ii in range(490, 510):
    
    fig = plt.figure(figsize=(12, 8))
    ax = plt.subplot(111, projection=mainproj)
    pcm = ax.pcolormesh(lon1, lat1, error1[ii,:,:], shading="auto",
                       cmap=plt.cm.RdYlGn_r, vmin=0., vmax=1., transform=datacrs)

    cb = plt.colorbar(pcm)

    ax.add_feature(coast, facecolor="#363636", edgecolor="k", zorder=5)
    ax.set_extent(dom)
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                          linewidth=.5, color="gray", alpha=1, linestyle="--")
    gl.top_labels = False
    gl.right_labels = False
    gl.xformatter = lon_formatter
    gl.yformatter = lat_formatter
    ax.set_title(f"Error field, {calendar.month_name[dates1[ii].month]} {dates1[ii].year}")
    
    figname = f"Canalus_error_L{resolution}_{dates1[ii].year}{str(dates1[ii].month).zfill(2)}"
    plt.savefig(os.path.join(resfigdir, figname))
    plt.close()